# Helpers

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

In [3]:
csv_dir = "../output/"

In [97]:
def parse_boundary(csv_path):
    data = pd.read_csv(csv_path, header=None)
    x, y, z = data[0], data[1], data[2]
    # x, y = (x + 1) / 2, (y + 1) / 2
    return x, y, z

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

def parse_csv_for_xy(path):
    # Read CSV assuming pairs of x, y coordinates (x1, y1, x2, y2, ...)
    data = pd.read_csv(path, header=None)
    
    # Convert all values to numeric, forcing errors to NaN (which makes them easier to handle)
    data = data.apply(pd.to_numeric, errors='coerce')

    # Flatten the data into x and y arrays
    x_vals = data.iloc[:, 0::2].values.flatten()  # Extract x values (every other column starting from 0)
    y_vals = data.iloc[:, 1::2].values.flatten()  # Extract y values (every other column starting from 1)

    return x_vals, y_vals

def get_original_points(s = 16):
    x = []
    y = []
    for j in range(s):
        for i in range(s):
            x.append((i / (s - 1)) * 2 - 1)
            y.append((j / (s - 1)) * 2 - 1)
    x = np.array(x)
    y = np.array(y)
    x = np.clip(x, 0, 1)
    y = np.clip(y, 0, 1)
    return x, y
        

def visualize_csv_output_vector(path, output_path="./solver_output_plot.png", original_point=None, xstart=0, ystart=0, xend=1, yend=1, surface=False):
    # Parse CSV to get x, y values
    x_vals, y_vals = parse_csv_for_xy(path)
    
    # Remove NaN values from the data
    valid_mask = ~np.isnan(x_vals) & ~np.isnan(y_vals)  # Create mask for non-NaN values in x and y
    x_vals = x_vals[valid_mask]  
    y_vals = y_vals[valid_mask] 
    
    assert len(x_vals) == len(y_vals), "x_vals and y_vals must have the same length"
    z_vals = np.zeros_like(x_vals)
    
    x = np.linspace(xstart, xend, 100)  # Increase resolution if necessary
    y = np.linspace(ystart, yend, 100)  # Increase resolution if necessary
    X, Y = np.meshgrid(x, y)
    
    # Interpolate the Z values onto the grid (if you want smooth surface interpolation)
    Z_interp = griddata((x_vals, y_vals), z_vals, (X, Y), method='linear')

    # Create a figure for plotting
    fig = go.Figure()

    # Add surface plot with interpolated Z values (if surface is true)
    if surface: 
        fig.add_trace(go.Surface(z=Z_interp, x=X, y=Y, colorscale='Viridis', showscale=True))

    # Add a scatter plot for the original points (if surface is false)
    if not surface:
        fig.add_trace(go.Scatter3d(x=x_vals, y=y_vals, z=z_vals, mode='markers',
            marker=dict(size=2, color='blue') 
        ))
        
    
    # Add boundary if provided
    if original_point is not None: 
        x_original, y_original = get_original_points()
        z_original = np.zeros_like(x_original) - 0.001
        fig.add_trace(go.Scatter3d(x=x_original, y=y_original, z=z_original, mode='markers',
            marker=dict(size=2, color='red')  
        ))

    # Update layout for better visualization
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        zaxis=dict(
            range=[-0.01, 0.01]  # Set the Z-axis range (adjust these values as needed)
        ),
        aspectmode='cube'  # Ensures that all axes are scaled equally
    ))

    # Show the plot and save to file
    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")
    
def visualize_csv_output_x(path, output_path="./solver_output_plot.png", original_point=False, xstart=0, ystart=0, xend=1, yend=1):
    data = pd.read_csv(path, header=None)
    x = np.linspace(xstart, xend, data.shape[1])
    y = np.linspace(ystart, yend, data.shape[0])
    X, Y = np.meshgrid(x, y)
    solved_X = data.values         

    # Create a mask for valid points
    valid_mask = ~np.isnan(solved_X)

    # Filter X, Y, Z using the mask
    X_valid = X[valid_mask]
    Y_valid = Y[valid_mask]
    solved_X_valid = solved_X[valid_mask]

    # Create a scatter plot for valid points
    fig = go.Figure()

    # Add a scatter plot for the valid points
    Z = np.zeros_like(solved_X_valid)
    fig.add_trace(go.Scatter3d(x=solved_X_valid, y=Y_valid, z=Z, mode='markers',
            marker=dict(size=2, color='blue')  # Customize marker appearance if needed
    ))
    
    if original_point: 
        x_original, y_original = get_original_points()
        z_original = np.zeros_like(x_original) - 0.001
        fig.add_trace(go.Scatter3d(x=x_original, y=y_original, z=z_original, mode='markers',
            marker=dict(size=2, color='red')  
        ))

    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ))

    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")


In [67]:
import plotly.graph_objects as go
def print_boundary_dirichlet(path=None, shape=None, dir=None): 
    if path is None: path = dir + shape + 'BoundaryDirichlet.csv'
    x, y, z = parse_boundary(path)
    fig = go.Figure()
    for i in range(len(x)):
        fig.add_trace(go.Scatter3d(
            x=[x[i], x[(i + 1) % len(x)]],
            y=[y[i], y[(i + 1) % len(y)]],
            z=[z[i], z[(i + 1) % len(z)]],
            mode='lines',
            line=dict(color='blue', width=2)
        ))
    fig.add_trace(go.Scatter3d(x=x,y=y,z=z,mode='markers',
        marker=dict(size=5, color='red', opacity=0.8)  # Color points
    ))

    fig.update_layout(title_text='3D Star Shape on a Tilted Plane',
                  scene=dict(
                      xaxis_title='X-axis',
                      yaxis_title='Y-axis',
                      zaxis_title='Z-axis'),
                  )

    fig.show()

In [26]:
def get_shape_csv(shape): 
    return shape + ".csv"
def get_boundaryD_csv(shape): 
    return shape + "BoundaryDirichlet.csv"

# Plot Deformed Rectangle

In [202]:
shape = "rectangle_displacement"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [203]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [204]:
visualize_csv_output_x(csv_dir + shape_csv, original_point=True) 

Figure saved as ./solver_output_plot.png


In [201]:
import numpy as np
import plotly.graph_objects as go

# Flatten the arrays
data = pd.read_csv(csv_dir + shape_csv, header=None)
solved_x = data.values
valid_mask = ~np.isnan(solved_x)
solved_X_valid = solved_x[valid_mask]

x_original, y_original = get_original_points(s = 16)
# x_original, y_original = np.meshgrid(x_original, y_original)

x_original = x_original.reshape(16, 16)
x_original = x_original[valid_mask]
print(x_original)
solved_X_valid

[0.06666667 0.2        0.33333333 0.46666667 0.6        0.73333333
 0.86666667 1.         0.06666667 0.2        0.33333333 0.46666667
 0.6        0.73333333 0.86666667 1.         0.06666667 0.2
 0.33333333 0.46666667 0.6        0.73333333 0.86666667 1.
 0.06666667 0.2        0.33333333 0.46666667 0.6        0.73333333
 0.86666667 1.         0.06666667 0.2        0.33333333 0.46666667
 0.6        0.73333333 0.86666667 1.         0.06666667 0.2
 0.33333333 0.46666667 0.6        0.73333333 0.86666667 1.
 0.06666667 0.2        0.33333333 0.46666667 0.6        0.73333333
 0.86666667 1.         0.06666667 0.2        0.33333333 0.46666667
 0.6        0.73333333 0.86666667]


array([0.0673427, 0.199679 , 0.332422 , 0.465166 , 0.599564 , 0.732032 ,
       0.866845 , 1.       , 0.0658678, 0.200125 , 0.334627 , 0.467648 ,
       0.599005 , 0.733914 , 0.86687  , 1.       , 0.0669325, 0.198812 ,
       0.335359 , 0.465971 , 0.599873 , 0.733041 , 0.866818 , 1.       ,
       0.0654518, 0.200044 , 0.334253 , 0.467314 , 0.599847 , 0.731713 ,
       0.867992 , 1.       , 0.0676252, 0.199154 , 0.334295 , 0.466636 ,
       0.60009  , 0.732645 , 0.866907 , 1.       , 0.0670446, 0.198807 ,
       0.33312  , 0.466694 , 0.601031 , 0.734402 , 0.867491 , 1.       ,
       0.0672787, 0.200524 , 0.332565 , 0.467108 , 0.599618 , 0.73141  ,
       0.866376 , 1.       , 0.066667 , 0.2      , 0.333333 , 0.466667 ,
       0.6      , 0.733333 , 0.866667 ])

In [163]:
min(x_vals - x_original)


-0.0021060000000000523

In [165]:
min(y_vals - y_original)


-0.0031003333333332606

In [161]:
x_original

array([0.06666667, 0.2       , 0.33333333, 0.46666667, 0.6       ,
       0.73333333, 0.86666667, 1.        , 0.06666667, 0.2       ,
       0.33333333, 0.46666667, 0.6       , 0.73333333, 0.86666667,
       1.        , 0.06666667, 0.2       , 0.33333333, 0.46666667,
       0.6       , 0.73333333, 0.86666667, 1.        , 0.06666667,
       0.2       , 0.33333333, 0.46666667, 0.6       , 0.73333333,
       0.86666667, 1.        , 0.06666667, 0.2       , 0.33333333,
       0.46666667, 0.6       , 0.73333333, 0.86666667, 1.        ,
       0.06666667, 0.2       , 0.33333333, 0.46666667, 0.6       ,
       0.73333333, 0.86666667, 1.        , 0.06666667, 0.2       ,
       0.33333333, 0.46666667, 0.6       , 0.73333333, 0.86666667,
       1.        , 0.06666667, 0.2       , 0.33333333, 0.46666667,
       0.6       , 0.73333333, 0.86666667])

In [160]:
x_vals

array([0.065957 , 0.200682 , 0.332834 , 0.466393 , 0.600029 , 0.733143 ,
       0.866304 , 1.       , 0.0665616, 0.200429 , 0.333983 , 0.466873 ,
       0.601339 , 0.734319 , 0.866526 , 1.       , 0.0666529, 0.200571 ,
       0.33308  , 0.464923 , 0.599984 , 0.733689 , 0.866313 , 1.       ,
       0.0662684, 0.202189 , 0.334418 , 0.469048 , 0.599661 , 0.731565 ,
       0.866078 , 1.       , 0.0666674, 0.198588 , 0.333006 , 0.466372 ,
       0.601175 , 0.732253 , 0.868288 , 1.       , 0.0670922, 0.200095 ,
       0.332204 , 0.464912 , 0.597894 , 0.731409 , 0.865139 , 1.       ,
       0.0662816, 0.200295 , 0.332323 , 0.465792 , 0.599963 , 0.733074 ,
       0.866684 , 1.       , 0.0666669, 0.2      , 0.333333 , 0.466667 ,
       0.6      , 0.733333 , 0.866667 ])